# Image summary and visual question answering

This notebooks shows how to generate image captions and use the visual question answering with [LAVIS](https://github.com/salesforce/LAVIS). 

The first cell is only run on google colab and installs the [ammico](https://github.com/ssciwr/AMMICO) package.

After that, we can import `ammico` and read in the files given a folder path.

In [1]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    # %pip install setuptools==61 -qqq
    # install ammico
    %pip install git+https://github.com/ssciwr/ammico.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [2]:
import ammico
from ammico import utils as mutils
from ammico import display as mdisplay
import ammico.summary as sm

In [3]:
# Here you need to provide the path to your google drive folder
# or local folder containing the images
images = mutils.find_files(
    path="data/",
    limit=10,
)

In [4]:
mydict = mutils.initialize_dict(images)

## Create captions for images and directly write to csv

Here you can choose between two models: "base" or "large". This will generate the caption for each image and directly put the results in a dataframe. This dataframe can be exported as a csv file.

The results are written into the columns `const_image_summary` - this will always be the same result (as always the same seed will be used). The column `3_non-deterministic summary` displays three different answers generated with different seeds, these are most likely different when you run the analysis again.

In [5]:
obj = sm.SummaryDetector(mydict)
summary_model, summary_vis_processors = obj.load_model(model_type="base")
# summary_model, summary_vis_processors = mutils.load_model("large")

  0%|          | 0.00/2.50G [00:00<?, ?B/s]

  0%|          | 4.01M/2.50G [00:00<01:28, 30.2MB/s]

  0%|          | 11.6M/2.50G [00:00<00:48, 55.2MB/s]

  1%|          | 17.3M/2.50G [00:00<00:51, 52.1MB/s]

  1%|          | 24.0M/2.50G [00:00<00:45, 58.0MB/s]

...

 94%|█████████▍| 1.27G/1.35G [00:07<00:00, 164MB/s]

 96%|█████████▌| 1.29G/1.35G [00:07<00:00, 181MB/s]

 98%|█████████▊| 1.32G/1.35G [00:07<00:00, 210MB/s]

100%|█████████▉| 1.35G/1.35G [00:08<00:00, 129MB/s]

100%|██████████| 1.35G/1.35G [00:08<00:00, 178MB/s]

In [6]:
for key in mydict:
    mydict[key] = sm.SummaryDetector(mydict[key]).analyse_image(
        summary_model=summary_model, summary_vis_processors=summary_vis_processors
    )

TypeError: analyse_image() got an unexpected keyword argument 'summary_model'

Convert the dictionary of dictionarys into a dictionary with lists:

In [7]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

Check the dataframe:

In [8]:
df.head(10)

,filename
0,102141_2_eng
1,102730_eng
2,106349S_por


Write the csv file:

In [9]:
df.to_csv("data_out.csv")

## Manually inspect the summaries

To check the analysis, you can inspect the analyzed elements here. Loading the results takes a moment, so please be patient. If you are sure of what you are doing.

`const_image_summary` - the permanent summarys, which does not change from run to run (analyse_image).

`3_non-deterministic summary` - 3 different summarys examples that change from run to run (analyse_image). 

In [10]:
analysis_explorer = mdisplay.AnalysisExplorer(mydict, identify="summary")
analysis_explorer.run_server(port=8055)

TypeError: __init__() got an unexpected keyword argument 'identify'

## Generate answers to free-form questions about images written in natural language. 

Set the list of questions as a list of strings:

In [11]:
list_of_questions = [
    "How many persons on the picture?",
    "Are there any politicians in the picture?",
    "Does the picture show something from medicine?",
]

Explore the analysis using the interface:

In [12]:
analysis_explorer = mdisplay.AnalysisExplorer(mydict, identify="summary")
analysis_explorer.run_server(port=8055)

TypeError: __init__() got an unexpected keyword argument 'identify'

## Or directly analyze for further processing
Instead of inspecting each of the images, you can also directly carry out the analysis and export the result into a csv. This may take a while depending on how many images you have loaded.

In [13]:
for key in mydict:
    mydict[key] = sm.SummaryDetector(mydict[key]).analyse_questions(list_of_questions)

FileNotFoundError: [Errno 2] No such file or directory: '102141_2_eng'

## Convert to dataframe and write csv
These steps are required to convert the dictionary of dictionarys into a dictionary with lists, that can be converted into a pandas dataframe and exported to a csv file.

In [14]:
outdict2 = mutils.append_data_to_dict(mydict)
df2 = mutils.dump_df(outdict2)

In [15]:
df2.head(10)

,filename
0,102141_2_eng
1,102730_eng
2,106349S_por


In [16]:
df2.to_csv("data_out2.csv")